In [2]:
%load_ext autoreload
%autoreload 2
import sys
import torch
sys.path.append("src")
from azure_util import AzureBlobStorage

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch

class Network(torch.nn.Module):
    
    def __init__(self):
        
        super().__init__()
        
        self._layers = torch.nn.Sequential(
            torch.nn.Linear(100, 200),
            torch.nn.ELU(),
            torch.nn.Linear(200, 2),
        )
        
    def forward(self, x):
        
        return self._layers(x)

In [7]:
network = Network()

In [13]:
import os
PROJECT_NAME = "test"
PROJECT_PATH = os.path.join("models", PROJECT_NAME)
VERSION = "1"
os.makedirs(PROJECT_PATH, exist_ok = True)
MODEL_PATH = os.path.join(PROJECT_PATH, VERSION)
os.makedirs(MODEL_PATH, exist_ok = True)


In [15]:
config = '''
name: "test_network"
platform: "pytorch_libtorch"
max_batch_size: 10
input [
  {
    name: "INPUT__0"
    data_type: TYPE_FP32
    dims: [100]
  }
]
output [
  {
    name: "OUTPUT__0"
    data_type: TYPE_FP32
    dims: [2]
  }
]

instance_group [
    {
        count: 1
        kind: KIND_GPU
    }
]
'''

f = open(os.path.join(PROJECT_PATH, "config.pbtxt"), "w")
f.write(config)
f.close()

In [ ]:
import torch
example = torch.rand(1, 100)
traced_script_module = torch.jit.trace(network, example)
traced_script_module.save(os.path.join(MODEL_PATH, "model.pt"))

In [ ]:
https://deeplearning.blob.core.windows.net/kserve

In [36]:
import os
azure_blob = AzureBlobStorage(
    connection_string=os.environ['AZURE_CHECKPOINT_CONNECTION_STRING'],
    container_name="kfserve2",#, os.environ['AZURE_CHECKPOINT_CONTAINER_NAME'],
)
azure_blob.upload_folder("context", "")

2023-05-14 10:28:09.080 95 azure.core.pipeline.policies.http_logging_policy INFO [on_request():510] Request URL: 'https://deeplearning.blob.core.windows.net/kfserve2/models/test/config.pbtxt'
Request method: 'PUT'
Request headers:
    'Content-Length': '294'
    'x-ms-blob-type': 'REDACTED'
    'If-None-Match': '*'
    'x-ms-version': 'REDACTED'
    'Content-Type': 'application/octet-stream'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.16.0 Python/3.10.9 (Linux-5.19.0-38-generic-x86_64-with-glibc2.27)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '0567ced2-f242-11ed-b0c5-0242ac110002'
    'Authorization': 'REDACTED'
A body is sent with the request
2023-05-14 10:28:09.408 95 azure.core.pipeline.policies.http_logging_policy INFO [on_response():549] Response status: 201
Response headers:
    'Content-Length': '0'
    'Content-MD5': 'REDACTED'
    'Last-Modified': 'Sun, 14 May 2023 10:28:09 GMT'
    'ETag': '"0x8DB5465E9C87072"'
    'Server':

In [23]:
import kserve

In [27]:
kserve.Storage.download("https://deeplearning.blob.core.windows.net/kserve/test", ".")

2023-05-14 07:59:03.977 95 root INFO [download():63] Copying contents of https://deeplearning.blob.core.windows.net/kserve/test to local
2023-05-14 07:59:03.978 95 root INFO [_download_azure_blob():348] Connecting to BLOB account: [deeplearning], container: [kserve], prefix: [test]
2023-05-14 07:59:03.979 95 root WARNING [_download_azure_blob():354] Azure credentials or shared access signature token not found, retrying anonymous access
2023-05-14 07:59:03.980 95 azure.core.pipeline.policies.http_logging_policy INFO [on_request():513] Request URL: 'https://deeplearning.blob.core.windows.net/kserve?restype=REDACTED&comp=REDACTED&prefix=REDACTED&delimiter=REDACTED'
Request method: 'GET'
Request headers:
    'x-ms-version': 'REDACTED'
    'Accept': 'application/xml'
    'User-Agent': 'azsdk-python-storage-blob/12.16.0 Python/3.10.9 (Linux-5.19.0-38-generic-x86_64-with-glibc2.27)'
    'x-ms-date': 'REDACTED'
    'x-ms-client-request-id': '31b5c7f6-f22d-11ed-b0c5-0242ac110002'
No body was at

ClientAuthenticationError: Server failed to authenticate the request. Please refer to the information in the www-authenticate header.
RequestId:5d23c3d5-c01e-004c-1e39-86a73a000000
Time:2023-05-14T07:59:04.1761368Z
ErrorCode:NoAuthenticationInformation
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>NoAuthenticationInformation</Code><Message>Server failed to authenticate the request. Please refer to the information in the www-authenticate header.
RequestId:5d23c3d5-c01e-004c-1e39-86a73a000000
Time:2023-05-14T07:59:04.1761368Z</Message></Error>

In [42]:
MODEL_NAME="test"
INPUT_FILE_PATH="input-triton.json"
INGRESS_HOST="test-predictor-default.default.svc.cluster.local"
INGRESS_PORT=80

s = f'''curl -v -H "Content-Type: application/json" \
    -X POST -d "@{INPUT_FILE_PATH}" \
    "http://{INGRESS_HOST}:{INGRESS_PORT}/v2/models/{MODEL_NAME}/infer"'''

In [45]:
s

'curl -v -H "Content-Type: application/json"     -X POST -d "@input-triton.json"     "http://test-predictor-default.default.svc.cluster.local:80/v2/models/test/infer"'

In [ ]:
curl -v -H "Content-Type: application/json" -X POST -d "@input-triton.json" "10.64.140.43:80/v2/models/test/infer"

In [ ]:
curl -v -H "Content-Type: application/json" -X POST -d "@input-triton.json" "127.0.0.1:8972/v2/models/test/infer"

In [ ]:
curl -v -H "Content-Type: application/json" -X POST -d "@data.json" "10.64.140.43:80/v2/models/test/infer"

In [ ]:
curl -v -H "Host: knative-local-gateway.istio-system.svc.cluster.local"-X POST -d "@data.json" "http://10.64.140.44:80/v2/models/test/infe" 